In [ ]:
import netgen.gui
from ngsolve import *
import netgen.geom2d as geom2dim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# CUSTOM geometry
#geo = geom2dim.SplineGeometry()

#vertices = [(0,0), (1,0), (1,1), (0,1)]
#p1, p2, p3, p4 = [geo.AppendPoint(x,y) for x,y in vertices]

# Dirichlet bc
# Right
#geo.Append(['line', p2, p3], bc='right')
# Left
#geo.Append(['line', p4, p1], bc='left')

# Neumann bc
# Bot
#geo.Append(['line', p1, p2], bc='bottom')
# Top
#geo.Append(['line', p1, p2], bc='top')
#mesh2 = Mesh(geo.GenerateMesh(maxh=2e-1))
#Draw(mesh2)

In [ ]:
#number_of_vertices = 4
mesh_size = 2e-1
mesh = Mesh(geom2dim.unit_square.GenerateMesh(maxh=mesh_size))
print("Number of vertices: ", mesh.nv)
print("Number of elements: ", mesh.ne)
Draw(mesh)

In [ ]:
Wl_errors = []
Wr_errors = []
Wd_errors = []
k = 2
hs = [0.5, 0.25, 0.2, 0.1, 0.05]
#hs = [0.05, 0.1, 0.2, 0.25, 0.5]
for i, mesh_size in enumerate(hs):
    mesh = Mesh(geom2dim.unit_square.GenerateMesh(maxh=mesh_size))
    fes = H1(mesh, order=k, dirichlet='right|left')

    # Set trial and test function
    u = fes.TrialFunction()
    v = fes.TestFunction()

    # Set right hand side
    rhs = LinearForm(fes)
    rhs += x*v*dx

    # The bilinear form
    A = BilinearForm(fes, symmetric=True)
    A += grad(u)*grad(v)*dx

    # Now assemble the system of equations
    with TaskManager():
        rhs.Assemble()
        A.Assemble()

    # Calculate the solution field (function)
    gf = GridFunction(fes)
    gf.vec.data = A.mat.Inverse(fes.FreeDofs(), inverse='sparsecholesky') * rhs.vec

    # Calculate temperature flux through left and right boundaries
    dudx = GridFunction(fes)
    dudx.Set(grad(gf)[0])
    #dudy = GridFunction(fes)
    #dudy.Set(grad(gf)[1])

    #Wl = Integrate(dudx, mesh, BND, region_wise=True)
    # BND... evaluate integral on boundary
    #print("Wl", Wl)
    Wl = Integrate(dudx, mesh, definedon=mesh.Boundaries('left'))
    Wr = Integrate(dudx, mesh, definedon=mesh.Boundaries('right'))
    Wd = Wl + Wr
    Wl_errors.append(Wl)
    Wr_errors.append(Wr)
    Wd_errors.append(Wd)

    pts = [(0,i) for i in np.arange(0, 1+mesh_size, mesh_size)]
    vals = [grad(gf)(mesh(x,y))[0] for x, y in pts]
    print("vals:", vals)
    # Integration using trapezoidal rule
    sum = 0
    for val1, val2 in zip(vals[:-1], vals[1:]):
        sum += (val1 + val2)/2*mesh_size

    # Output to console
    print("Wl=", Wl)
    print("Wr=", Wr)
    print("Total flux through dirichlet boundaries:\nWd= ", Wd)
    print("Trapezoidal:", sum)
    print('-'*30)

Draw(gf, mesh, 'grid_function')
Draw(grad(gf), mesh, 'gradient_gf')

In [ ]:
# Analytical (exact) values
Wr_exact = -1/3
Wl_exact = 1/6
Wd_exact = Wr_exact + Wl_exact

print('-'*30)
print("Analytical (exact) value:")
print("Wl_exact=", Wl_exact)
print("Wr_exact=", Wr_exact)
print("Wd_exact=", Wd_exact)

In [ ]:
print(Wl_errors)
print(Wr_errors)
Wl_errors = np.array(Wl_errors) - Wl_exact
Wr_errors = np.array(Wr_errors) - Wr_exact
Wd_errors = np.array(Wd_errors) - Wd_exact

In [ ]:
print(Wl_errors)
print(Wr_errors)

plt.semilogy(hs, Wl_errors, '-xr', label='Wl')
plt.semilogy(hs, Wr_errors, '-pb', label='Wr')
plt.semilogy(hs, Wd_errors, '-xg', label='Wd')
plt.suptitle("k={}".format(k))
plt.xlabel("h... mesh size")
plt.grid()
plt.legend()
plt.show()